In [1]:
import tskit
import pyslim
import msprime
import dendropy
import glob
import numpy as np
import pandas as pd
import warnings
import functools
import argparse
import operator
import os


In [2]:
parser = argparse.ArgumentParser(description='Gets stats from unioned tree sequence')
parser.add_argument('rand_id', type=str)
parser.add_argument('rep', type=str)
parser.add_argument('win_size', type=lambda x: int(float(x)))
parser.add_argument('sample_size', type=int)
parser.add_argument('coords_dict', type=str, help="String of a dictionary with padded and non-padded start and ends of the chromosomic region. Assumes one chromosome only!")
parser.add_argument('--seed', type=int, default=8991, required=False)

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=8991, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
args = vars(parser.parse_args(["GDVOP9EMEV6ZLEG","0","1000000","10","{'chr': 'chr12', 'start': 60000000, 'end': 70000000, 'padded_start': 60000000, 'padded_end': 70000000}"]))
coords_dict = eval(args['coords_dict'])
out_path = "../../output/"

In [4]:
coords_dict = eval(args['coords_dict'])

In [5]:
# Loading tree sequence and list with populations
recap_mut_path = f"{out_path}{args['rand_id']}/{args['rand_id']}_rep{args['rep']}.union.recap.mut.trees"
pops_path = f"{out_path}{args['rand_id']}/{args['rand_id']}_rep{args['rep']}.pops"
assert os.path.exists(recap_mut_path) and os.path.exists(pops_path), f"Trees file or .pops file does not exist for {args['rand_id']}_{args['rep']}"
recap_tsu = pyslim.load(recap_mut_path)
with open(pops_path, "r") as f:
    pops = eval(f.readline())

rng = np.random.default_rng(args['seed'])

In [6]:
rng = np.random.default_rng(args['seed'])
# getting contemporary samples
# note the time of "contemporary" samples varies bc of differences in generation times
# TODO: sample individuals not nodes
pop_samples = [recap_tsu.samples(population_id=i+1) for i in range(len(pops))]
contemp_time = [np.min(recap_tsu.tables.nodes.time[samples]) for samples in pop_samples]
contemp_samples = [rng.choice(pop_samples[pid][recap_tsu.tables.nodes.time[pop_samples[pid]] == contemp_time[pid]], args["sample_size"], replace=False) for pid in range(len(pop_samples))]

In [18]:
sample_sets = contemp_samples
n = [len(s) for s in sample_sets]

twoway = [(x,y) for x in range(len(pops)) for y in range(len(pops)) if x>=y]
fourway = [(twoway[xx],twoway[yy]) for xx in range(len(twoway)) for yy in range(len(twoway)) if xx>yy]


In [24]:
dxy_funs = {
    (i,j): lambda x: x[i] * (n[j] - x[j]) / (n[i] * (n[j] - (i == j)))
    for (i,j) in twoway
}

In [27]:
def g(a, b, c, d):
    print(a,b,c,d)
    f1 = dxy_funs[(a, b)]
    f2 = dxy_funs[(c, d)]
    return lambda x: f1(x) * f2(x)

In [28]:
recap_tsu.sample_count_stat(contemp_samples, g(0,0,0,0), output_dim=1)

0 0 0 0


ValueError: object of too small depth for desired array

In [ ]:
# pull out the trees and get a densiplot

In [ ]:
# plot the residuals of those correlations against branch lengths/N or sth? that would help us there.